In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from PIL import Image
from IPython.core.display import HTML
from functools import partial

In [6]:
%reload_ext autoreload
%autoreload 2
from engine.utils_nav import ProgramGenerator, ProgramInterpreter
from prompts.gqa import create_prompt

In [ ]:

from engine.step_interpreters_nav import parse_step

eval_ = """EVAL0=EVAL(expr="'STOP' if {LOC0} > 0 else 'NAVIGATE'")"""
stop = """STOP0=STOP(var=EVAL0)"""
# str_2 = """ANSWER2=EVAL(expr="'yes' if {ANSWER0} + {ANSWER1} > 0 else 'no'")"""
dict_ = parse_step(eval_, partial=False)
dict_

In [71]:
question = """ go to [0,10,3]"""
answer= """NAV0=NAVIGATE(image=START, goal='[0,10,3]')
LOC0=LOCATE(pos=NAV0)
EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
STOP0=STOP(var=EVAL0)
"""

image = Image.open('../assets/startnav.png')
image.thumbnail((640,640),Image.Resampling.LANCZOS)
init_state = dict(
    IMAGE=image.convert('RGB')
)

prompter = partial(create_prompt,method='all')
generator = ProgramGenerator(prompter=prompter, debug=True)

interpreter = ProgramInterpreter(dataset='pointnav')
prog,_ = generator.generate(dict(question=question), answer) #fake answer
print(prog)

Registering NAVIGATE step
Registering DETECT step
Registering STOP step
Registering LOCATE step
Registering EVAL step
NAV0=NAVIGATE(image=START, goal='[0,10,3]')
LOC0=LOCATE(pos=NAV0)
EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
STOP0=STOP(var=EVAL0)


In [74]:
result, prog_state = interpreter.execute(prog,\
                                        init_state,\
                                        inspect=False)
result

{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F2E35BEAF50>}
NAVIGATE
{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F2E35BEAF50>, 'NAV0': 0.2}
LOCATE
{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F2E35BEAF50>, 'NAV0': 0.2, 'LOC0': True}
EVAL
{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F2E35BEAF50>, 'NAV0': 0.2, 'LOC0': True, 'EVAL0': 'STOP'}
STOP


True